In [1]:
import pandas as pd
import random

import geopandas as gpd 
from shapely.geometry import Point, Polygon
import h3

import plotly.express as px

import json

In [2]:
data = pd.DataFrame(columns=['Latitude', 'Longitude'])

In [3]:
min_latitude = 43.5
max_latitude = 47.1
min_longitude = 6.6
max_longitude = 11.5

In [4]:
num_points = 1000000  # Cantidad de puntos a generar

lat = []
long = []

for _ in range(num_points):
    latitude = random.uniform(min_latitude, max_latitude)
    longitude = random.uniform(min_longitude, max_longitude)
    
    lat.append(latitude)
    long.append(longitude)

In [5]:
df = pd.DataFrame(list(zip(lat, long)),
               columns =['latitude', 'longitude'])

In [6]:
italia = gpd.read_file("data/Com01012023_g_WGS84.shp")
italia.geometry = italia.geometry.to_crs(epsg = 4326)
italia.head()

,COD_RIP,DEN_RIP,Shape_Leng,Shape_Area,geometry
0,1,Nord-Ovest,2.330183e+06,5.792958e+10,"MULTIPOLYGON (((9.85132 44.02340, 9.85122 44.0..."
1,2,Nord-Est,2.327765e+06,6.238509e+10,"MULTIPOLYGON (((10.48080 44.18949, 10.48069 44..."
2,3,Centro,2.010203e+06,5.801865e+10,"MULTIPOLYGON (((13.45526 40.78719, 13.45309 40..."
3,4,Sud,2.517097e+06,7.377795e+10,"MULTIPOLYGON (((15.80124 39.69743, 15.80108 39..."
4,5,Isole,2.775538e+06,4.991778e+10,"MULTIPOLYGON (((12.55987 35.50930, 12.55969 35..."


In [7]:
# Create a geometry column from lng & lat
df['geometry'] = df.apply(lambda x: Point(float(x.longitude), float(x.latitude)), axis=1)

# Create a GeoDataFrame from art and verify the type
df_points = gpd.GeoDataFrame(df, crs = italia.crs, geometry = df.geometry)
print(type(df_points))

df_points.head()

<class 'geopandas.geodataframe.GeoDataFrame'>


,latitude,longitude,geometry
0,44.560293,7.196442,POINT (7.19644 44.56029)
1,46.595971,6.797809,POINT (6.79781 46.59597)
2,44.443197,7.931607,POINT (7.93161 44.44320)
3,46.415057,8.258995,POINT (8.25900 46.41506)
4,44.263331,8.692100,POINT (8.69210 44.26333)


In [8]:
df_italia = gpd.sjoin(df_points, italia, op = 'within')
print(df_italia.shape)

/home/oecorrechag/.local/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


(612382, 8)


In [9]:
# df_italia = df_italia[df_italia['DEN_RIP'] == 'Nord-Ovest']
# df_italia.head()

,latitude,longitude,geometry,index_right,COD_RIP,DEN_RIP,Shape_Leng,Shape_Area
0,44.560293,7.196442,POINT (7.19644 44.56029),0,1,Nord-Ovest,2.330183e+06,5.792958e+10
2,44.443197,7.931607,POINT (7.93161 44.44320),0,1,Nord-Ovest,2.330183e+06,5.792958e+10
6,44.353628,7.840122,POINT (7.84012 44.35363),0,1,Nord-Ovest,2.330183e+06,5.792958e+10
7,44.654045,7.587566,POINT (7.58757 44.65405),0,1,Nord-Ovest,2.330183e+06,5.792958e+10
9,45.352362,7.979367,POINT (7.97937 45.35236),0,1,Nord-Ovest,2.330183e+06,5.792958e+10


In [10]:
df_italia['latitude'].min(), df_italia['latitude'].max()

(43.77943444589514, 46.631824373573004)

In [11]:
df_italia['longitude'].min(), df_italia['longitude'].max()

(6.631585445831979, 11.42294280863007)

In [12]:
df_italia['DEN_RIP'].unique()

array(['Nord-Ovest'], dtype=object)

In [13]:
df_italia['DEN_RIP'].value_counts()

DEN_RIP
Nord-Ovest    376266
Name: count, dtype: int64

# Agregar las ciudades

In [14]:
# # save
# df.to_csv('data/df.csv', encoding = 'utf-8-sig', index = False)

# Generar los H3

In [15]:
def get_h3_shape(row):
    h = h3.geo_to_h3(row["latitude"], row["longitude"], 9)
    return h

def get_geometry(h) :
    vertices = h3.h3_to_geo_boundary(h,  geo_json=True)
    polygon = Polygon(vertices)
    return polygon

# def get_geometry(h) :
#     h = h3.h3_to_geo_boundary(h, geo_json=True)
#     return Polygon(h)

In [16]:
df_italia["h3"] = df_italia.apply(get_h3_shape, axis=1)
df_italia["geometry"] = df_italia["h3"].apply(get_geometry)
df_italia.head()

,latitude,longitude,geometry,index_right,COD_RIP,DEN_RIP,Shape_Leng,Shape_Area,h3
0,44.560293,7.196442,"POLYGON ((7.19431 44.55942, 7.19455 44.55764, ...",0,1,Nord-Ovest,2.330183e+06,5.792958e+10,891f9a89a3bffff
2,44.443197,7.931607,"POLYGON ((7.92737 44.44378, 7.92760 44.44200, ...",0,1,Nord-Ovest,2.330183e+06,5.792958e+10,891f9a3089bffff
6,44.353628,7.840122,"POLYGON ((7.83864 44.35554, 7.83887 44.35376, ...",0,1,Nord-Ovest,2.330183e+06,5.792958e+10,891f9a3a9d3ffff
7,44.654045,7.587566,"POLYGON ((7.58385 44.65493, 7.58409 44.65315, ...",0,1,Nord-Ovest,2.330183e+06,5.792958e+10,891f9ab824bffff
9,45.352362,7.979367,"POLYGON ((7.97821 45.35287, 7.97844 45.35111, ...",0,1,Nord-Ovest,2.330183e+06,5.792958e+10,891f98016bbffff


In [31]:
df_italia = gpd.GeoDataFrame(df_italia, crs = italia.crs, geometry = df_italia.geometry)
df_italia = df_italia.loc[:,['geometry', 'h3', 'COD_RIP']]
print(type(df_italia))
df_italia.head()

<class 'geopandas.geodataframe.GeoDataFrame'>


,geometry,h3,COD_RIP
1,"POLYGON ((8.70271 45.72135, 8.70293 45.71959, ...",891f98a4027ffff,1
24,"POLYGON ((9.33777 45.41386, 9.33797 45.41208, ...",891f9956aafffff,1
36,"POLYGON ((9.10932 44.97942, 9.10952 44.97764, ...",891f9bb32d3ffff,1
41,"POLYGON ((7.02920 45.89853, 7.02945 45.89679, ...",891f9c45d13ffff,1
46,"POLYGON ((8.96436 44.73538, 8.96456 44.73360, ...",891f9b17517ffff,1


In [32]:
# save
df_italia.to_file('data/df_italia.geojson', driver='GeoJSON')  

In [ ]:
with open('data/df_italia2.geojson', 'r') as infile:
    geo_json = json.load(infile)

In [ ]:
geo_json

In [ ]:
fig = px.choropleth(df_italia2, geojson=geo_json, locations='COD_RIP', color='COD_RIP',
                           color_continuous_scale="Viridis",
                           range_color=(0, 12),
                        #    scope="usa",
                           labels={'unemp':'unemployment rate'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
df_italia['DEN_RIP'].unique()

In [ ]:
type(df_italia)

In [ ]:
import pyproj

In [ ]:
df_italia.to_crs(pyproj.CRS.from_epsg(4326), inplace=True)

In [ ]:
df_italia.head()

In [ ]:

fig = (
    px.scatter_mapbox(
        df_italia,
        lat="latitude",
        lon="longitude",
        color="COD_RIP",
        hover_data=["COD_RIP", "DEN_RIP"],
    )
    .update_traces(marker={"size": 10})
    .update_layout(
        mapbox={
            "style": "open-street-map",
            "zoom": 5,
            "layers": [
                {
                    "source": json.loads(df_italia.geometry2.to_json()),
                    "below": "traces",
                    "type": "line",
                    "color": "purple",
                    "line": {"width": 1.5},
                }
            ],
        },
        margin={"l": 0, "r": 0, "t": 0, "b": 0},
    )
)
fig.show()

In [ ]:
print('Ok_')